<a href="https://colab.research.google.com/github/Karagiannis-Giorg/A-new-perspective-to-Smart-Fridges/blob/main/Simulation/Simulation_Y_Y.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install roboflow
!pip install ultralytics==8.3.40
!pip install streamlit ngrok opencv-python-headless
!pip install pyngrok
!ngrok config add-authtoken 2rXJVXzr26fTTIW0Qj1torNd6Il_iiVaeFHCJtw5zki9ubVc

from google.colab import drive
drive.mount('/content/gdrive')
ROOT_DIR = '/content/gdrive/My Drive/Georgios_Karagiannis/Code'

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.1/83.1 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 19.9 MB/s eta 0:00:00
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.11.0.86
    Uninstalling opencv-python-headless-4.11.0.86:
      Successfully uninstalled opencv-python-headless-4.11.0.86
  Attempting uninstall: idna
    Found existing installation: idna 3.10
    Uninstalling idna-3.10:
      Successfully uninstalled idna-3.10
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 898.5/898.5 kB 28.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 111.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 87.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 50.6 MB/s eta 0:0

In [ ]:
%%writefile smart_fridge_app.py
import streamlit as st
import numpy as np
import glob
import cv2
import random
from google.colab.patches import cv2_imshow
import os
import shutil
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from ultralytics import YOLO
import json

# Ρύθμιση YOLO
def setup_yolo_level1():
    model1 = YOLO("/content/gdrive/My Drive/Georgios_Karagiannis/Code/Level1_train/detect/train/weights/best.pt")
    return model1

def setup_yolo_level2():
    model2 = YOLO("/content/gdrive/My Drive/Georgios_Karagiannis/Code/Level2_train/detect/train/weights/best.pt")
    return model2

# Ανάλυση Εικόνας
def analyze_image(image, temperature, humidity, yolo_model_level1, yolo_model_level2):
    yolo_results_level1 = yolo_model_level1(image)
    fruit_classes = [yolo_model_level1.names[int(cls)] for cls in yolo_results_level1[0].boxes.cls]

    if "Tomato" in fruit_classes:
        yolo_results_level2 = yolo_model_level2(image)
        ripeness_classes = [yolo_model_level2.names[int(cls)] for cls in yolo_results_level2[0].boxes.cls]
        ripeness_message = []

        for ripeness in ripeness_classes:
            if ripeness == "over-ripe":
                ripeness_message.append("Over-ripe tomatoes detected. Discard them immediatley before they start to effect the other fruits.")
            elif ripeness == "ripe":
                if 7 <= temperature <= 10 and humidity > 70:
                    ripeness_message.append("Ripe tomatoes detected. They are edible for up to 4 days. If you want your tomatoes to last longer, you should store them somewhere cooler.")
                elif temperature < 7:
                    ripeness_message.append("You should put your ripe tomatoes somewhere warmer to avoid chilling injuries.")
                else:
                    ripeness_message.append("YYour tomatoes will ripen quickly, but you should bear in mind that they are susceptible to pathogens.")
            elif ripeness == "half-ripe":
                if 8 <= temperature <= 10 and humidity > 60:
                    ripeness_message.append("Half-ripe tomatoes detected. They are not edible but can be stored for up to 7 days untill they are edible.If you want to ripe them more slower , you should store them somewhere cooler and with less humidity")
                elif temperature < 8:
                    ripeness_message.append("Your rippening prosses will be stoped and your tomatoes will be susceptible to chilling injuries.")
                else:
                    ripeness_message.append("Your tomatoes will ripen quickly, but you should bear in mind that they are susceptible to pathogens.")
            elif ripeness == "unripe":
                if 8 <= temperature <= 12 and humidity > 50:
                    ripeness_message.append("Unripe tomatoes detected. They are not edible but can be stored for up to 10 days untill they are edible.If you want to ripe them more slower , you should store them somewhere cooler and with less humidity")
                elif temperature < 8:
                    ripeness_message.append("Your rippening prosses will be stoped and your tomatoes will be susceptible to chilling injuries.")
                else:
                    ripeness_message.append("Your tomatoes will ripen quickly, but you should bear in mind that they are susceptible to pathogens.")
        return ripeness_message
    else:
        remaining_items = [item for item in fruit_classes if item != "Tomato"]
        return f"No tomatoes detected. Items in the fridge: {', '.join(remaining_items)}"

# Streamlit UI
def main():
    # Κεντρικός τίτλος
    st.markdown(
        "<h1 style='text-align: center;'>Welcome back</h1>",
        unsafe_allow_html=True,
    )
    # Εισαγωγή δεδομένων χρήστη
    temperature = st.number_input(
        "Temperature (°C)", value=8.0, min_value=-10.0, max_value=15.0, step=1.0
    )
    humidity = st.number_input(
        "Humidity (%)", value=75.0, min_value=0.0, max_value=100.0, step=1.0
    )

    st.markdown(
        "<h3 style='text-align: center;'>What would you like to do?</h3>",
        unsafe_allow_html=True,
    )

    # Αρχικοποίηση Session State για την εικόνα
    if "selected_image" not in st.session_state:
        simulation_image_path = "/content/gdrive/MyDrive/Georgios_Karagiannis/Code/simulation"
        image_files = glob.glob(os.path.join(simulation_image_path, "*.[jp][pn]g"))
        if not image_files:
            st.error("No images found in the test directory.")
            st.stop()
        st.session_state.selected_image = random.choice(image_files)

    selected_image = st.session_state.selected_image

    if st.button("See what's inside"):
        # Εκτέλεση YOLO για ανάλυση περιεχομένων ψυγείου
        yolo_model_level1 = setup_yolo_level1()

        # Διαβάζουμε την εικόνα
        img = cv2.imread(selected_image)

        # Εκτελούμε το μοντέλο YOLO στην εικόνα
        results = yolo_model_level1([img])

        # Σχεδιάζουμε τα αποτελέσματα πάνω στην εικόνα
        for result in results:
            result.save(filename="output.jpg")  # Αποθηκεύουμε την έξοδο σε αρχείο

        # Εμφάνιση νέας ανάλυσης
        st.success("Analysis Complete!")
        st.image("output.jpg", caption="Detected Items in the Fridge", use_container_width=True)
        fruit_classes = [yolo_model_level1.names[int(cls)] for cls in results[0].boxes.cls]
        fruits_counts = {fruits: fruit_classes.count(fruits) for fruits in set(fruit_classes)}

        st.write("Contents of the fridge:")
        for fruits, count in fruits_counts.items():
            st.write(f"- {count} {fruits}(s)")

    if st.button("See the maturity"):

        # Εκτέλεση YOLO για ανάλυση ωριμότητας
        yolo_model_level1 = setup_yolo_level1()
        yolo_model_level2 = setup_yolo_level2()

        # Διαβάζουμε την εικόνα
        img = cv2.imread(selected_image)

        # Εκτελούμε το μοντέλο YOLO στην εικόνα
        results = yolo_model_level2([img])

        # Σχεδιάζουμε τα αποτελέσματα πάνω στην εικόνα
        for result in results:
            result.save(filename="output.jpg")  # Αποθηκεύουμε την έξοδο σε αρχείο

        # Ανάλυση ωριμότητας
        ripeness_analysis = analyze_image(selected_image, temperature, humidity, yolo_model_level1, yolo_model_level2)

        # Εμφάνιση νέας ανάλυσης
        st.success("Analysis Complete!")
        st.image("output.jpg", caption="Ripeness of tomatoes detected", use_container_width=True)

        # Εμφάνιση ανάλυσης ωριμότητας
        if isinstance(ripeness_analysis, list):
            st.write("Ripeness Results:")
            for msg in ripeness_analysis:
                st.write("- " + msg)
        else:
            st.write(ripeness_analysis)

if __name__ == "__main__":
    main()

Writing smart_fridge_app.py


In [ ]:
from pyngrok import ngrok
import os

# Εκκίνηση Streamlit server
os.system("streamlit run smart_fridge_app.py &")

# Σύνδεση με ngrok
public_url = ngrok.connect(8501)  # Η θύρα πρέπει να είναι αριθμός χωρίς `port=`
print(f"Public URL: {public_url}")

Public URL: NgrokTunnel: "https://fbfa-34-125-49-191.ngrok-free.app" -> "http://localhost:8501"


# Αξιολόγηση του Pipeline

In [ ]:
from roboflow import Roboflow
rf = Roboflow(api_key="F357b98Kl2koA8RqsGzW")
project = rf.workspace("simulation-sv1vx").project("simulation-repenning")
version = project.version(2)
dataset = version.download("yolov11")

rf = Roboflow(api_key="F357b98Kl2koA8RqsGzW")
project = rf.workspace("simulation-sv1vx").project("simulation-dsrze")
version = project.version(2)
dataset = version.download("yolov11")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Simulation-repenning-2 in yolov11:: 100%|██████████| 52/52 [00:00<00:00, 632.18it/s]


loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Simulation-2 in yolov11:: 100%|██████████| 52/52 [00:00<00:00, 2893.31it/s]


In [ ]:
import streamlit as st
import numpy as np
import glob
import cv2
import random
from google.colab.patches import cv2_imshow
import os
import shutil
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from ultralytics import YOLO
import json

def evaluate_yolo(model_path, dataset_path):
    model = YOLO(model_path)
    results = model.val(data=dataset_path)  # Αξιολόγηση
    return results.box.maps  # Επιστροφή mAP τιμών

# Υπολογισμός συνολικής ακρίβειας
def compute_overall_accuracy(yolo_metrics_level1, yolo_metrics_level2):
    yolo_map_level1 = yolo_metrics_level1[1]*100 # mAP@50 του YOLO
    yolo_map_level2 = yolo_metrics_level2[1]*100
    print(f"YOLO mAP Level1: {yolo_map_level1}")
    print(f"YOLO mAP Level2: {yolo_map_level2}")
    overall_accuracy = (yolo_map_level1 + yolo_map_level2) / 2  # Μέσος όρος
    return overall_accuracy

yolo_results_level1=evaluate_yolo("/content/gdrive/My Drive/Georgios_Karagiannis/Code/Level1_train/detect/train/weights/best.pt", "/content/Simulation-2/data.yaml")
yolo_results_level2=evaluate_yolo("/content/gdrive/My Drive/Georgios_Karagiannis/Code/Level2_train/detect/train/weights/best.pt", "/content/Simulation-repenning-2/data.yaml")

overall_accuracy = compute_overall_accuracy(yolo_results_level1, yolo_results_level2)
print(f"Overall Accuracy: {overall_accuracy}")

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Ultralytics 8.3.40 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLO11n summary (fused): 238 layers, 2,583,712 parameters, 0 gradients, 6.3 GFLOPs


100%|██████████| 755k/755k [00:00<00:00, 24.6MB/s]
val: Scanning /content/Simulation-2/valid/labels... 23 images, 0 backgrounds, 0 corrupt: 100%|██████████| 23/23 [00:00<00:00, 512.71it/s]

val: New cache created: /content/Simulation-2/valid/labels.cache



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:02<00:00,  1.21s/it]


                   all         23        104      0.952      0.967      0.993      0.923
                 Apple         13         15      0.929          1      0.995      0.946
                Banana         10         10      0.958        0.9      0.978      0.825
                  Lime          7          7       0.98          1      0.995      0.938
                Orange          5          5      0.932          1      0.995      0.976
                 Peach          6          6      0.858          1      0.995      0.947
                  Pear          3          3      0.964          1      0.995      0.865
           Pomegranate          9          9          1      0.864      0.995      0.957
                Tomato         22         49          1      0.972      0.995      0.931
Speed: 7.6ms preprocess, 9.8ms inference, 0.0ms loss, 15.9ms postprocess per image
Results saved to runs/detect/val
Ultralytics 8.3.40 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
Y

val: Scanning /content/Simulation-repenning-2/valid/labels... 23 images, 1 backgrounds, 0 corrupt: 100%|██████████| 23/23 [00:00<00:00, 2468.50it/s]

val: New cache created: /content/Simulation-repenning-2/valid/labels.cache



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  2.34it/s]


                   all         23         49      0.915      0.898      0.955      0.835
             half-ripe         15         16      0.852      0.812      0.879      0.754
             over-ripe          4          4      0.946          1      0.995      0.862
                  ripe         10         10          1      0.833      0.986      0.907
                unripe         17         19      0.862      0.947      0.961      0.817
Speed: 0.2ms preprocess, 13.0ms inference, 0.0ms loss, 1.6ms postprocess per image
Results saved to runs/detect/val2
YOLO mAP Level1: 82.4628276116648
YOLO mAP Level2: 86.23333333333333
Overall Accuracy: 84.34808047249908
